In [1]:
import numpy as np
import abc

# Baseclass

In [2]:
class Node(abc.ABC):
    
    @abc.abstractmethod
    def forward(self):
        """Feed Forward"""
    
    @abc.abstractmethod
    def backward(self, dout):
        """Back Propagate
        Inputs:
            dout: upstream gradient"""

    @property
    def grads(self):
        return self._grads

#  Variable

In [3]:
class Variable(Node):
    
    def __init__(self, val):
        self._v = val
    
    def forward(self):
        return self._v
    
    def backward(self, dout):
        self._grads = dout

# Binary

In [4]:
class Add(Node):
    
    def __init__(self, a, b):
        self._a = a
        self._b = b
    
    def forward(self):
        v_a = self._a.forward()
        v_b = self._b.forward()
        # self._local_da = 1.
        # self._local_db = 1.
        return v_a + v_b
    
    def backward(self, dout):
        self._grads = [dout, dout]
        self._a.backward(dout)
        self._b.backward(dout)


class Mul(Node):
    
    def __init__(self, a, b):
        self._a = a
        self._b = b
    
    def forward(self):
        v_a = self._a.forward()
        v_b = self._b.forward()
        self._local_da = v_b
        self._local_db = v_a
        return v_a * v_b
    
    def backward(self, dout):
        da = self._local_da * dout
        db = self._local_db * dout
        self._grads = [da, db]
        self._a.backward(da)
        self._b.backward(db)

# Unary

In [5]:
class Inv(Node):
    
    def __init__(self, a):
        self._a = a
    
    def forward(self):
        val = self._a.forward()
        self._local_grads = - 1. / val**2
        return 1. / val
    
    def backward(self, dout):
        self._grads = self._local_grads * dout
        self._a.backward(self._grads)
        

class Exp(Node):
    
    def __init__(self, a):
        self._a = a
    
    def forward(self):
        val = self._a.forward()
        self._local_grads = np.exp(val)
        return self._local_grads
    
    def backward(self, dout):
        self._grads = self._local_grads * dout
        self._a.backward(self._grads)

# Test

$$f(w,x) = \frac{1}{1 + \exp[-(w_1x_1 + w_2x_2 + b)]}$$

In [6]:
# Init variables
w1, w2, x1, x2, b = [Variable(float(i)) for i in [2, -3, -1, -2, -3]]

# build graph
logit = Add(Add(Mul(w1, x1), Mul(w2, x2)), b)
f = Inv(Add(Variable(1), Exp(Mul(logit, Variable(-1)))))

# Eval
print('Values: \n', logit.forward(), f.forward())
# BP
f.backward(1.0)
print('Gradients: ')
print(', '.join('{:.2f}'.format(v.grads) for v in [w1, w2, x1, x2, b]))

Values: 
 1.0 0.73105857863
Gradients: 
-0.20, -0.39, 0.39, -0.59, 0.20


# A bug

$$f = wx_1 + wx_2 = w(x_1+x_2)$$

In [7]:
w, x1, x2 = [Variable(float(i)) for i in [5, 1, 2]]
f = Add(Mul(w, x1), Mul(w, x2))
print('Value: \n', f.forward())
f.backward(1.)
print('Gradients: \n', [v.grads for v in [w, x1, x2]])

Value: 
 15.0
Gradients: 
 [2.0, 5.0, 5.0]
